In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jul  9 11:30:59 2020

@author: Tamara
"""

# prep: 
    #pip install selenium
    #pip install webdriver_manager
    
#%% import different stuff
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import requests
import time
import xlrd,xlwt
import os
from datetime import datetime, date, timedelta
from dateutil.relativedelta import *

# for dropdown: 
from selenium.webdriver.support.ui import Select

import numpy  as np
from webdriver_manager.chrome import ChromeDriverManager

#%% online options

tradeSession = requests.Session()
options = Options()
options.incognito = True
options.headless = False

#%% login 

url = 'https://search-proquest-com.ezproxy1.lib.asu.edu'

# other option: 
## https://search-proquest-com.ezproxy1.lib.asu.edu/ethnicnewswatch/socialsciences/fromDatabasesLayer/advanced
                                                                              
#%% define login process
def theLogin():

	driver.get(url)
    
	time.sleep(1.5+np.random.uniform() ) 
	username = driver.find_element_by_name("username")                        
	username.send_keys(usr)
                                                                              
	time.sleep(2+np.random.uniform()) 
	passwd = driver.find_element_by_name("password")                           

	passwd.send_keys(password)
	passwd.send_keys(Keys.ENTER)

	time.sleep(1+np.random.uniform()) 
	driver.get(url)
	time.sleep(2.5+np.random.uniform())
    
	try:
		iframe = driver.find_elements_by_tag_name('iframe')[0]
		driver.switch_to.frame(iframe)
		cookie_button = driver.find_element_by_xpath("xpath=//a[contains(text(),'Agree and Proceed')]")
		cookie_button.click()
	except:
		pass

	try:
		cookie_button = driver.find_element_by_xpath("/html/body/div[8]/div[1]/div/div[2]/div[2]/a[1]")
		cookie_button.click()
	except:
		pass
#%% define writing to excel
    
def write_to_xl(thedate,title):
	outsheet.write(index,0,thedate)
	outsheet.write(index,1,thesource)
	outsheet.write(index,2,title)
    
#%% define query 
def search(query):
	global index
	driver.get(url)
	driver.find_element_by_name("searchTerm").send_keys(query+Keys.ENTER)
	time.sleep(3+np.random.uniform()) #6

	try:
		thelink = driver.find_element_by_id("citationDocTitleLink").click()
		time.sleep(1+np.random.uniform()) #2
	except:
		return

	while True:

		title = driver.find_element_by_id("documentTitle").text
		title = title.replace(" ","_")

		symbols = '''!"#$%&\'()*+,-./:;?@"'''

		for s in symbols:
			title = title.replace(s, "")

		title = title[:50]
		time.sleep(5+np.random.uniform())
        
#click for full text here, if not then download PDF
		try: 
			driver.find_element_by_link_text('Full text').click()                 # added click for full text 
			time.sleep(1+np.random.uniform() ) 
			thedata = driver.find_element_by_id("fullTextZone").text
		except: 
			try: 
				thedata = "see pdf"
				driver.find_element_by_id('downloadPDFLink').click()
				time.sleep(.5+np.random.uniform()) 
			except:
				pass
            
#try get the date             
		try:                                                                      # Fix this because not always the same date format
			thedate = driver.find_elements_by_class_name("titleAuthorETC")[1].text.split("]")[1].split(":")[0]
		except: 
			thedate = None 
            
		write_to_xl(thedate,title)
		index += 1


		with open(folderName+"/"+title+".txt","w") as f:
			f.write(thedata)
		try:
			driver.find_element_by_id("nextLink").click()
			time.sleep(.5+np.random.uniform()) #2
		except:
			break
#%% folder for output
folderName = "Output - " + datetime.now().strftime("%d-%m-%Y") + " - Kate"

try:
	os.mkdir(folderName)
except FileExistsError:
	pass

#%% output 
outworkbook = xlwt.Workbook()
outsheet = outworkbook.add_sheet("result")
thetime = str(int(time.time()))

index = 0

# Login
usr = 'cseigel'
password = 'Santa!Fe12'

# Search terms - to edit if want to change the source 
searchwords = "(immigration OR immigrant OR refugee) AND (illegal OR undocumented)"
thesource = 'New York Times (1923-Current file)'
# other options: "News from Indian Country (1985-present)", "Akwesasne Notes (1975-1989)", "Seminole Tribune (1984-1989)", "The Jewish Press (1977-1989)" , 
query = searchwords + " AND (pub("+str(thesource)+"))" 
#query = "(immigration OR immigrant OR refugee) AND (illegal OR undocumented) and PUB(New York Times)"
secondpart = " NOT rtype.exact(review) AND stype(Newspapers) AND stype.exact('Historical Newspapers')"



In [ ]:
#### Go through time selection as loop 

if os.path.exists("curdate.txt"):
	with open("curdate.txt",'r') as f:
		newdate = f.read()
		print(newdate)
	end_date = datetime.strptime(newdate, '%Y%m%d')
else:
	end_date = datetime.strptime("19891231", '%Y%m%d')
# Put as end day one day after the day you want! 
    
start_date = datetime.strptime("19850101", '%Y%m%d')

deltaone = relativedelta(days=+1)
# Below you can edit to have more or less days in the search term. 
delta = relativedelta(days=+30)

while start_date <= end_date:
    
	options = Options()
	options.incognito = True
	options.headless = False
	driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
    
	theLogin()
	end_date -= deltaone
	curdate = end_date.strftime("%Y%m%d")
	end_date -= delta
	lengthdate = end_date.strftime("%Y%m%d")
	newquery = query + " AND PD("+str(lengthdate)+"-"+str(curdate)+")" + secondpart
    
	print(newquery)
	search(newquery)
    
	outworkbook.save(thetime+".xls")
	driver.quit()

	to_store = end_date.strftime("%Y%m%d")

	with open("curdate.txt",'w') as f:
		f.write(to_store)

	time.sleep(5+np.random.uniform())

os.remove("curdate.txt")
######################

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/tanhakate/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


19881224
 


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: use options instead of chrome_options


(immigration OR immigrant OR refugee) AND (illegal OR undocumented) AND (pub(New York Times (1923-Current file))) AND PD(19881123-19881223) NOT rtype.exact(review) AND stype(Newspapers) AND stype.exact('Historical Newspapers')


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/tanhakate/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


...

(immigration OR immigrant OR refugee) AND (illegal OR undocumented) AND (pub(New York Times (1923-Current file))) AND PD(19880521-19880620) NOT rtype.exact(review) AND stype(Newspapers) AND stype.exact('Historical Newspapers')


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/tanhakate/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


 
(immigration OR immigrant OR refugee) AND (illegal OR undocumented) AND (pub(New York Times (1923-Current file))) AND PD(19880420-19880520) NOT rtype.exact(review) AND stype(Newspapers) AND stype.exact('Historical Newspapers')


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/tanhakate/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


 
(immigration OR immigrant OR refugee) AND (illegal OR undocumented) AND (pub(New York Times (1923-Current file))) AND PD(19880320-19880419) NOT rtype.exact(review) AND stype(Newspapers) AND stype.exact('Historical Newspapers')
